In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm

from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier

from datasets import Dataset
from armin import AlgorithmicRecourseExplainer, ArminExplainer
from armin.baselines import ImputationAlgorithmicRecourseExplainer, RobustAlgorithmicRecourseExplainer
from armin.missing_helper import MissingGenerator, SingleImputer, MultipleImputer
from armin._utils import sign_agreement

In [2]:
N = 100
DATASETS = ['f', 'e', 's', 'w']
MODELS = ['L', 'M', 'F']
MAX_N_MISSING = 3
MAX_CHANGE_NUM = 4
CONFIDENCE = 0.75
CONFIDENCES = [0.4, 0.5, 0.6, 0.7, 0.8]
N_SAMPLING = 100
N_SAMPLINGS = [100, 200, 300, 400, 500]

## Experiment 1. Baseline Comparison under MCAR Situation

In [3]:
def run_comparison(
        N=3, 
        dataset='f', 
        model='L',
        max_n_missing=1,
        cost_type='TLPS',
        max_change_num=4,
        confidences=[0.5],
        n_sampling=100,
        time_limit=60,
        verbose=True,
    ):
    np.random.seed(0)

    if model=='L':
        clf = LogisticRegression(penalty='l2', C=1.0, solver='liblinear')
    elif model=='M':
        clf = MLPClassifier(hidden_layer_sizes=(30,), max_iter=500, activation='relu', alpha=0.0001)
    elif model=='F':
        clf = RandomForestClassifier(n_estimators=50, max_leaf_nodes=8, class_weight='balanced')

    D = Dataset(dataset=dataset)
    X_tr, X_ts, y_tr, y_ts = D.get_dataset(split=True, test_size=0.25)
    clf = clf.fit(X_tr, y_tr)
    X = X_ts[clf.predict(X_ts)==1][:N]
    N = X.shape[0]

    mg = MissingGenerator(D.feature_types, D.feature_categories)
    si_mean = SingleImputer(imputer_type='mean', feature_types=D.feature_types, feature_categories=D.feature_categories).fit(X_tr)
    si_knn = SingleImputer(imputer_type='knn', feature_types=D.feature_types, feature_categories=D.feature_categories).fit(X_tr)
    si_mice = SingleImputer(imputer_type='mice', feature_types=D.feature_types, feature_categories=D.feature_categories).fit(X_tr)
    mi = MultipleImputer(n_sampling=n_sampling, imputer_type='mice', feature_types=D.feature_types, feature_categories=D.feature_categories).fit(X_tr)

    ar = AlgorithmicRecourseExplainer(clf, X_tr, y_tr, **D.params)
    iar_mean = ImputationAlgorithmicRecourseExplainer(clf, si_mean, X_tr, y_tr, **D.params)
    iar_knn = ImputationAlgorithmicRecourseExplainer(clf, si_knn, X_tr, y_tr, **D.params)
    iar_mice = ImputationAlgorithmicRecourseExplainer(clf, si_mice, X_tr, y_tr, **D.params)
    rar = RobustAlgorithmicRecourseExplainer(clf, mi, X_tr, y_tr, **D.params)
    armin = ArminExplainer(clf, mi, X_tr, y_tr, **D.params)

    methods = ['mean', 'knn', 'mice', 'robust', 'armin']
    results = {
        'n_missing': [],
        'method': [],
        'confidence': [],
        'feasible': [], 
        'valid': [], 
        'cost': [], 
        'relative_cost': [], 
        'time': [], 
        'probability_target': [], 
        'sign_agreement': [], 
        'y_init': [], 
    }
    keys = results.keys()

    def update_result_dict(action, n_missing, method, confidence, a_opt, c_opt):
        action['n_missing'] = n_missing 
        action['method'] = method 
        action['sign_agreement'] = sign_agreement(action['action'], a_opt)
        action['relative_cost'] = action['cost'] / c_opt 
        action['confidence'] = confidence
        for key in keys: results[key].append(action[key])

    for n in tqdm(range(N)):
        if verbose:
            print('# Instance', n+1)

        action = ar.extract(X[n], max_change_num=max_change_num, cost_type=cost_type)
        if not action['solved']: continue
        a_opt = action['action']; c_opt = action['cost']; 
        update_result_dict(action, 0, 'optimal', 1, a_opt, c_opt)
        if verbose:
            print('## Optimal action before missing')
            print(ar.getActionObject(action))

        for n_missing in tqdm(range(1, max_n_missing+1), leave=False):
            
            x_missing = mg.mask_instance(X[n], n_missing=n_missing)
            if verbose:
                print('## Action after missing (n_missing = {})'.format(n_missing))

            for method, bar in zip(methods, [iar_mean, iar_knn, iar_mice, rar]):
                action = bar.extract(x_missing, max_change_num=max_change_num, cost_type=cost_type)
                action = bar.updateActionDicts(X[n], action)
                update_result_dict(action, n_missing, method, 1, a_opt, c_opt)
                if verbose:
                    print('### Baseline ({})'.format(method))
                    print(bar.getActionObject(action))

            for confidence in confidences:
                action = armin.extract(x_missing, confidence=confidence, max_change_num=max_change_num, cost_type=cost_type, time_limit=time_limit)
                action = armin.updateActionDicts(X[n], action)
                update_result_dict(action, n_missing, 'armin', confidence, a_opt, c_opt)
                if verbose:
                    print('### ARMIN (conf. = {})'.format(confidence))
                    print(armin.getActionObject(action))

    pd.DataFrame(results).to_csv('./res/{}/{}_{}.csv'.format(model, dataset, cost_type), index=False)


In [ ]:
for model in MODELS:
    for dataset in DATASETS:
        run_comparison(
            N=N, 
            dataset=dataset, 
            model=model,
            max_n_missing=MAX_N_MISSING,
            cost_type='TLPS',
            max_change_num=MAX_CHANGE_NUM,
            confidences=[CONFIDENCE],
            n_sampling=N_SAMPLING,
            verbose=False,
        )

## Experiment 2. Comparison under MAR and MNAR Situations 

In [5]:
def run_gmc(
        N=3, 
        situation='MAR',
        cost_type='TLPS',
        max_change_num=4,
        confidences=[0.5],
        n_sampling=100,
        verbose=True,
    ):
    np.random.seed(0)

    clf = LogisticRegression(penalty='l2', C=1.0, solver='liblinear')
    D = Dataset(dataset='g')
    X_tr, X_ts, y_tr, y_ts = D.get_dataset(split=True, test_size=0.25)
    clf = clf.fit(X_tr, y_tr)

    if situation=='MAR':
        X_old = X_ts[X_ts[:, 1] > np.median(X_tr[:, 1])]
    elif situation=='MNAR':
        X_old = X_ts[X_ts[:, 4] > np.median(X_tr[:, 4])]
    X = X_old[clf.predict(X_old)==1][:N]
    N = X.shape[0]

    mg = MissingGenerator(D.feature_types, D.feature_categories)
    si_mean = SingleImputer(imputer_type='mean', feature_types=D.feature_types, feature_categories=D.feature_categories).fit(X_tr)
    si_knn = SingleImputer(imputer_type='knn', feature_types=D.feature_types, feature_categories=D.feature_categories).fit(X_tr)
    si_mice = SingleImputer(imputer_type='mice', feature_types=D.feature_types, feature_categories=D.feature_categories).fit(X_tr)
    mi = MultipleImputer(n_sampling=n_sampling, imputer_type='mice', feature_types=D.feature_types, feature_categories=D.feature_categories).fit(X_tr)

    ar = AlgorithmicRecourseExplainer(clf, X_tr, y_tr, **D.params, quantile=(0.01, 0.99))
    iar_mean = ImputationAlgorithmicRecourseExplainer(clf, si_mean, X_tr, y_tr, **D.params, quantile=(0.01, 0.99))
    iar_knn = ImputationAlgorithmicRecourseExplainer(clf, si_knn, X_tr, y_tr, **D.params, quantile=(0.01, 0.99))
    iar_mice = ImputationAlgorithmicRecourseExplainer(clf, si_mice, X_tr, y_tr, **D.params, quantile=(0.01, 0.99))
    rar = RobustAlgorithmicRecourseExplainer(clf, mi, X_tr, y_tr, **D.params, quantile=(0.01, 0.99))
    armin = ArminExplainer(clf, mi, X_tr, y_tr, **D.params, quantile=(0.01, 0.99))

    methods = ['mean', 'knn', 'mice', 'robust', 'armin']
    results = {
        'n_missing': [],
        'method': [],
        'confidence': [],
        'feasible': [], 
        'valid': [], 
        'cost': [], 
        'relative_cost': [], 
        'time': [], 
        'probability_target': [], 
        'sign_agreement': [], 
        'y_init': [], 
    }
    keys = results.keys()

    def update_result_dict(action, n_missing, method, confidence, a_opt, c_opt):
        action['n_missing'] = n_missing 
        action['method'] = method 
        action['sign_agreement'] = sign_agreement(action['action'], a_opt)
        action['relative_cost'] = action['cost'] / c_opt 
        action['confidence'] = confidence
        for key in keys: results[key].append(action[key])

    for n in tqdm(range(N)):
        if verbose:
            print('# Instance', n+1)

        action = ar.extract(X[n], max_change_num=max_change_num, cost_type=cost_type)
        if (not action['solved']) or (not action['valid']): continue
        a_opt = action['action']; c_opt = action['cost']; 
        update_result_dict(action, 0, 'optimal', 1, a_opt, c_opt)
        if verbose:
            print('## Optimal action before missing')
            print(ar.getActionObject(action))
            
        x_missing = mg.mask_instance(X[n], n_missing=1, m_=[4])
        if verbose:
            print('## Action after missing')

        for method, bar in zip(methods, [iar_mean, iar_knn, iar_mice, rar]):
            action = bar.extract(x_missing, max_change_num=max_change_num, cost_type=cost_type)
            action = bar.updateActionDicts(X[n], action)
            update_result_dict(action, 1, method, 1, a_opt, c_opt)
            if verbose:
                print('### Baseline ({})'.format(method))
                print(bar.getActionObject(action))

        for confidence in confidences:
            action = armin.extract(x_missing, confidence=confidence, max_change_num=max_change_num, cost_type=cost_type)
            action = armin.updateActionDicts(X[n], action)
            update_result_dict(action, 1, 'armin', confidence, a_opt, c_opt)
            if verbose:
                print('### ARMIN (conf. = {})'.format(confidence))
                print(armin.getActionObject(action))

    pd.DataFrame(results).to_csv('./res/L/g_{}_{}.csv'.format(situation, cost_type), index=False)


In [6]:
for situation in ['MAR', 'MNAR']:
    run_gmc(
        N=N, 
        situation=situation,
        cost_type='TLPS',
        max_change_num=MAX_CHANGE_NUM,
        confidences=CONFIDENCES,
        n_sampling=N_SAMPLING,
        verbose=False,
    )

## Experiment 3. Confidence Path Analysis

In [7]:
def run_path(
        N=1,
        situation='MAR',
        cost_type='TLPS',
        max_change_num=4,
        n_sampling=100,
        verbose=True,
    ):
    np.random.seed(0)

    clf = LogisticRegression(penalty='l2', C=1.0, solver='liblinear')
    D = Dataset(dataset='g')
    X_tr, X_ts, y_tr, y_ts = D.get_dataset(split=True, test_size=0.25)
    clf = clf.fit(X_tr, y_tr)

    if situation=='MAR':
        X_old = X_ts[X_ts[:, 1] > np.median(X_tr[:, 1])]
    elif situation=='MNAR':
        X_old = X_ts[X_ts[:, 4] > np.median(X_tr[:, 4])]
    X = X_old[clf.predict(X_old)==1]

    y_prob_inits = clf.predict_proba(X)[:, 0]
    y_prob_sorted = np.argsort(y_prob_inits)
    X = X[y_prob_sorted]
    y_prob_inits = y_prob_inits[y_prob_sorted]
    Ns = np.sort(np.random.choice(np.arange(X.shape[0]), N, replace=False))

    mg = MissingGenerator(D.feature_types, D.feature_categories)
    mi = MultipleImputer(n_sampling=n_sampling, imputer_type='mice', feature_types=D.feature_types, feature_categories=D.feature_categories).fit(X_tr)

    ar = AlgorithmicRecourseExplainer(clf, X_tr, y_tr, **D.params, quantile=(0.01, 0.99))
    armin = ArminExplainer(clf, mi, X_tr, y_tr, **D.params, quantile=(0.01, 0.99))

    results = {
        'n': [],
        'n_missing': [],
        'method': [],
        'confidence': [],
        'feasible': [], 
        'valid': [], 
        'cost': [], 
        'relative_cost': [], 
        'time': [], 
        'probability_target': [], 
        'sign_agreement': [], 
        'y_prob_init': [], 
    }
    keys = results.keys()

    def update_result_dict(action, n, n_missing, method, confidence, a_opt, c_opt, y_prob_init):
        action['n'] = n 
        action['n_missing'] = n_missing 
        action['method'] = method 
        action['sign_agreement'] = sign_agreement(action['action'], a_opt)
        action['relative_cost'] = action['cost'] / c_opt 
        action['confidence'] = confidence
        action['y_prob_init'] = y_prob_init
        for key in keys: results[key].append(action[key])

    for n in tqdm(Ns):
        if verbose:
            print('# Instance', n+1)
            
        action = ar.extract(X[n], max_change_num=max_change_num, cost_type=cost_type)
        if (not action['solved']) or (not action['valid']): continue
        a_opt = action['action']; c_opt = action['cost']; 
        update_result_dict(action, n, 0, 'optimal', 1, a_opt, c_opt, y_prob_inits[n])
        if verbose:
            print('## Optimal action before missing')
            print(ar.getActionObject(action))
            
        x_missing = mg.mask_instance(X[n], n_missing=1, m_=[4])
        if verbose:
            print('## Action after missing')

        path = armin.confidence_path(x_missing, max_change_num=max_change_num, cost_type=cost_type)
        for (confidence, action) in path:
            action = armin.updateActionDicts(X[n], action)
            update_result_dict(action, n, 1, 'armin', confidence, a_opt, c_opt, y_prob_inits[n])
            if verbose:
                print('### ARMIN (conf. = {})'.format(confidence))
                print(armin.getActionObject(action))

    pd.DataFrame(results).to_csv('./res/L/path_g_{}_{}.csv'.format(situation, cost_type), index=False)


In [8]:
for situation in ['MAR', 'MNAR']:
    run_path(
        N=N, 
        situation=situation,
        cost_type='TLPS',
        max_change_num=MAX_CHANGE_NUM,
        n_sampling=N_SAMPLING,
        verbose=False,
    )

# Experiment 4. Sensitivity Analyses

In [9]:
def run_sensitivity(
        N=3, 
        dataset='f', 
        n_missing=1,
        cost_type='TLPS',
        max_change_num=4,
        confidences=[0.5],
        n_samplings=[100],
        verbose=True,
    ):
    np.random.seed(0)

    clf = LogisticRegression(penalty='l2', C=1.0, solver='liblinear')
    D = Dataset(dataset=dataset)
    X_tr, X_ts, y_tr, y_ts = D.get_dataset(split=True, test_size=0.25)
    clf = clf.fit(X_tr, y_tr)
    X = X_ts[clf.predict(X_ts)==1][:N]
    N = X.shape[0]

    results = {
        'n_missing': [],
        'method': [],
        'confidence': [],
        'n_sampling': [],
        'feasible': [], 
        'valid': [], 
        'cost': [], 
        'relative_cost': [], 
        'time': [], 
        'probability_target': [], 
        'sign_agreement': [], 
        'y_init': [], 
    }
    keys = results.keys()

    mg = MissingGenerator(D.feature_types, D.feature_categories)
    mi = MultipleImputer(imputer_type='mice', feature_types=D.feature_types, feature_categories=D.feature_categories).fit(X_tr)

    ar = AlgorithmicRecourseExplainer(clf, X_tr, y_tr, **D.params, quantile=(0.01, 0.99))
    armin = ArminExplainer(clf, mi, X_tr, y_tr, **D.params, quantile=(0.01, 0.99))

    def update_result_dict(action, n_missing, method, confidence, n_sampling, a_opt, c_opt):
        action['n_missing'] = n_missing 
        action['method'] = method 
        action['sign_agreement'] = sign_agreement(action['action'], a_opt)
        action['relative_cost'] = action['cost'] / c_opt 
        action['confidence'] = confidence
        action['n_sampling'] = n_sampling
        for key in keys: results[key].append(action[key])

    for n in tqdm(range(N)):
        if verbose:
            print('# Instance', n+1)

        action = ar.extract(X[n], max_change_num=max_change_num, cost_type=cost_type)
        if not action['solved']: continue
        a_opt = action['action']; c_opt = action['cost']; 
        update_result_dict(action, 0, 'optimal', 1, 0, a_opt, c_opt)
        if verbose:
            print('## Optimal action before missing')
            print(ar.getActionObject(action))

        x_missing = mg.mask_instance(X[n], n_missing=n_missing)
        if verbose:
            print('## Action after missing (n_missing = {})'.format(n_missing))

        for n_sampling in n_samplings:
            armin.imputer_.n_sampling = n_sampling
            for confidence in confidences:
                action = armin.extract(x_missing, confidence=confidence, max_change_num=max_change_num, cost_type=cost_type)
                action = armin.updateActionDicts(X[n], action)
                update_result_dict(action, n_missing, 'armin', confidence, n_sampling, a_opt, c_opt)
                if verbose:
                    print('### ARMIN (conf. = {} | n_sampling = {})'.format(confidence, n_sampling))
                    print(armin.getActionObject(action))

    pd.DataFrame(results).to_csv('./res/L/sens_{}_{}_{}.csv'.format('confidence' if len(n_samplings) == 1 else ('sampling' if len(confidences) == 1 else 'both'), dataset, cost_type), index=False)


In [10]:
for dataset in DATASETS:
    run_sensitivity(
        N=N, 
        dataset=dataset, 
        n_missing=2,
        cost_type='TLPS',
        max_change_num=MAX_CHANGE_NUM,
        confidences=CONFIDENCES,
        n_samplings=[N_SAMPLING],
        verbose=False,
    )

In [11]:
for dataset in DATASETS:
    run_sensitivity(
        N=N, 
        dataset=dataset, 
        n_missing=2,
        cost_type='TLPS',
        max_change_num=MAX_CHANGE_NUM,
        confidences=[CONFIDENCE],
        n_samplings=N_SAMPLINGS,
        verbose=False,
    )